# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

##出現提示欄進行授權

os.chdir('/content/drive/My Drive/ML2/HW7/') #切換該目錄
os.listdir() #確認目錄內容

# Download link 1
!gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Mounted at /content/drive
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb
To: /content/drive/My Drive/ML2/HW7/hw7_data.zip
100% 9.57M/9.57M [00:00<00:00, 255MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Wed Apr 27 02:27:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|     

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [ ]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.1 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 55.4 MB/s 
     |████████████████████████████████| 3.3 MB 52.1 MB/s 


## Import Packages

In [ ]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast, BertConfig

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(101)

In [ ]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = False

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer




 

In [ ]:
# config = BertConfig.from_pretrained("hfl/chinese-macbert-large", hidden_dropout_prob=0.2)
# model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-large").to(device)
model1 = BertForQuestionAnswering.from_pretrained("saved_model_roberta_devless_less_0.1_0.1_400_3_0.842").to(device)
model2 = BertForQuestionAnswering.from_pretrained("saved_model_roberta_devless_less_0.1_0.1_400_3_0.836").to(device)
model3 = BertForQuestionAnswering.from_pretrained("saved_model_roberta_devless_less_0.1_0.1_400_3_0.831").to(device)
# model4 = BertForQuestionAnswering.from_pretrained("saved_model_macbert_devless0.1_0.1_440_3_0.883").to(device)
# model5 = BertForQuestionAnswering.from_pretrained("saved_model_macbert_devless0.1_0.1_384_3_0.841").to(device)
# model6 = BertForQuestionAnswering.from_pretrained("saved_model_macbert_devless0.1_0.1_384_4_0.850").to(device)

tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")
##-------------------------------------------------------------------------------------------------------------------
# config = BertConfig.from_pretrained("hfl/chinese-macbert-large", hidden_dropout_prob=0.4)
# model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-large", config=config).to(device)
# tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")
##-------------------------------------------------------------------------------------------------------------------
# from transformers import BertTokenizer, T5ForConditionalGeneration, Text2TextGenerationPipeline
# tokenizer = BertTokenizer.from_pretrained("uer/t5-small-chinese-cluecorpussmall")
# model = T5ForConditionalGeneration.from_pretrained("uer/t5-small-chinese-cluecorpussmall")
# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [ ]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]


train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
dev_questions_final, dev_paragraphs_final = read_data("hw7_dev.json")

print("Train questions before :", len(train_questions))
print("Train paragraphs before :", len(train_paragraphs))
print("Dev questions before :", len(dev_questions))
print("Dev paragraphs before :", len(dev_paragraphs))
# counter = 0
for i in range(len(dev_questions)):
  if int(dev_questions[i]["paragraph_id"]) < 1400:
    train_questions += [dev_questions[i]]
    dev_questions_final.remove(dev_questions[i])
    # counter+=1
  
train_paragraphs += dev_paragraphs[:1400]
dev_paragraphs_final = dev_paragraphs[1400:]
print("Train questions after :", len(train_questions))
print("Train paragraphs after :", len(train_paragraphs))
print("Dev questions  after :", len(dev_questions_final))
print("Dev paragraphs after :", len(dev_paragraphs_final))

# print(counter) #3137

for number in range(31691, 35644):
  train_questions[number]["id"] = int(number)
  train_questions[number]["paragraph_id"] = 10524 + int(train_questions[number]["paragraph_id"])


for number in range(177):
  dev_questions_final[number]['id'] = int(number)
  dev_questions_final[number]['paragraph_id'] = int(dev_questions_final[number]['paragraph_id']) - 1400

# for i in range(31691, 34827):
  
# print(train_questions[34111])
# print(train_paragraphs[11268])



test_questions, test_paragraphs = read_data("hw7_test.json")

Train questions before : 31690
Train paragraphs before : 10524
Dev questions before : 4131
Dev paragraphs before : 1490
Train questions after : 35644
Train paragraphs after : 11924
Dev questions  after : 177
Dev paragraphs after : 90


## Tokenize Data

In [ ]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions_final], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs_final, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [ ]:
from random import randint
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 60
        self.max_paragraph_len = 400
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 32

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]
        
        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            if answer_start_token == None or answer_end_token == None:
              print(question["answer_start"])
              print(question["answer_end"])
              print(answer_start_token)
              print(answer_end_token) 
              print("--------------")
              answer_start_token = 0
              answer_end_token = 1
            # A single window is obtained by slicing the portion of paragraph containing the answer
            shift_num = randint(-50, +50)
            shift_num_right = randint(0, +50)
            shift_num_left = randint(-50, 0)
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            

            
            if paragraph_start >= abs(shift_num) and (len(tokenized_paragraph) - self.max_paragraph_len)>= abs(shift_num):
              paragraph_start += shift_num
              paragraph_end = paragraph_start + self.max_paragraph_len
            
            else:
              paragraph_end = paragraph_start + self.max_paragraph_len

            # paragraph_end = paragraph_start + self.max_paragraph_len #sample code
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token
            
        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list), question["id"]

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions_final, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 4

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [ ]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    # print(data[0].shape)
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        # print("start_index", start_index)
        # print("end_index", end_index)
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob and (end_index - start_index) < 30 and end_index > start_index:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    

    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [ ]:
from transformers import get_linear_schedule_with_warmup
num_epoch = 5
validation = True
logging_step = 100
accumulation_steps = 8
learning_rate = 5e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

total_steps = len(train_loader) * num_epoch
warm_up_ratio = 0.01
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = (warm_up_ratio * total_steps)//accumulation_steps, num_training_steps = total_steps//accumulation_steps) 

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

step = 1
for epoch in range(num_epoch):
    
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
      
      # Load all data into GPU
      data = [i.to(device) for i in data]
      
      # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
      # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
      output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

      # Choose the most probable start position / end position
      start_index = torch.argmax(output.start_logits, dim=1)
      end_index = torch.argmax(output.end_logits, dim=1)
      
      # Prediction is correct only if both start_index and end_index are correct
      train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
      train_loss += output.loss / accumulation_steps
      
      if fp16_training:
          accelerator.backward(output.loss)
      else:
          output.loss.backward()
      
      if (step) % accumulation_steps == 0:             # Wait for several backward steps
        optimizer.step()                         # Now we can do an optimizer step
        model.zero_grad()                         # Reset gradients tensors
        scheduler.step()
      
      step += 1

      ##### TODO: Apply linear learning rate decay #####
      # optimizer.param_groups[0]['lr'] -= learning_rate / total_steps
      
      # Print training loss and accuracy over past logging step
      if step % logging_step == 0:
          print(f"Learning Rate {optimizer.param_groups[0]['lr']:.8f} | Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
          train_loss = train_acc = 0

      if validation and step % 870 == 0 :
          print("Evaluating Dev Set ...")
          model.eval()
          with torch.no_grad():
              dev_acc = 0
              for i, data in enumerate(tqdm(dev_loader)):
                  output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                        attention_mask=data[2].squeeze(dim=0).to(device))
                  # prediction is correct only if answer text exactly matches
                  dev_acc += evaluate(data, output) == dev_questions_final[data[3]]["answer_text"]
              print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
              print("Saving Model ...")
              model_save_dir = f"saved_model_roberta_devless_less_0.1_0.1_400_{epoch+1}_{dev_acc / len(dev_loader):.3f}" 
              model.save_pretrained(model_save_dir)
          model.train()
      

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
# print("Saving Model ...")
# model_save_dir = "saved_model" 
# model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/8911 [00:00<?, ?it/s]

Learning Rate 0.00001091 | Epoch 1 | Step 100 | loss = 0.747, acc = 0.000
Learning Rate 0.00002182 | Epoch 1 | Step 200 | loss = 0.592, acc = 0.050
Learning Rate 0.00003364 | Epoch 1 | Step 300 | loss = 0.274, acc = 0.325
Learning Rate 0.00004455 | Epoch 1 | Step 400 | loss = 0.172, acc = 0.502
Learning Rate 0.00004994 | Epoch 1 | Step 500 | loss = 0.161, acc = 0.590
Learning Rate 0.00004983 | Epoch 1 | Step 600 | loss = 0.128, acc = 0.630
Learning Rate 0.00004971 | Epoch 1 | Step 700 | loss = 0.126, acc = 0.645
Learning Rate 0.00004960 | Epoch 1 | Step 800 | loss = 0.108, acc = 0.683
Evaluating Dev Set ...


  0%|          | 0/177 [00:00<?, ?it/s]

Validation | Epoch 1 | acc = 0.757
Saving Model ...
Learning Rate 0.00004948 | Epoch 1 | Step 900 | loss = 0.123, acc = 0.632
Learning Rate 0.00004937 | Epoch 1 | Step 1000 | loss = 0.125, acc = 0.650
Learning Rate 0.00004926 | Epoch 1 | Step 1100 | loss = 0.116, acc = 0.662
Learning Rate 0.00004915 | Epoch 1 | Step 1200 | loss = 0.089, acc = 0.745
Learning Rate 0.00004903 | Epoch 1 | Step 1300 | loss = 0.112, acc = 0.667
Learning Rate 0.00004892 | Epoch 1 | Step 1400 | loss = 0.097, acc = 0.715
Learning Rate 0.00004880 | Epoch 1 | Step 1500 | loss = 0.102, acc = 0.715
Learning Rate 0.00004869 | Epoch 1 | Step 1600 | loss = 0.109, acc = 0.680
Learning Rate 0.00004858 | Epoch 1 | Step 1700 | loss = 0.091, acc = 0.712
Evaluating Dev Set ...


...

Validation | Epoch 5 | acc = 0.836
Saving Model ...
Learning Rate 0.00000210 | Epoch 5 | Step 42700 | loss = 0.010, acc = 0.972
Learning Rate 0.00000199 | Epoch 5 | Step 42800 | loss = 0.008, acc = 0.985
Learning Rate 0.00000188 | Epoch 5 | Step 42900 | loss = 0.008, acc = 0.980
Learning Rate 0.00000177 | Epoch 5 | Step 43000 | loss = 0.010, acc = 0.975
Learning Rate 0.00000165 | Epoch 5 | Step 43100 | loss = 0.008, acc = 0.982
Learning Rate 0.00000154 | Epoch 5 | Step 43200 | loss = 0.008, acc = 0.977
Learning Rate 0.00000142 | Epoch 5 | Step 43300 | loss = 0.006, acc = 0.985
Learning Rate 0.00000131 | Epoch 5 | Step 43400 | loss = 0.006, acc = 0.977
Learning Rate 0.00000120 | Epoch 5 | Step 43500 | loss = 0.003, acc = 0.982
Evaluating Dev Set ...


  0%|          | 0/177 [00:00<?, ?it/s]

Validation | Epoch 5 | acc = 0.847
Saving Model ...
Learning Rate 0.00000109 | Epoch 5 | Step 43600 | loss = 0.002, acc = 0.993
Learning Rate 0.00000097 | Epoch 5 | Step 43700 | loss = 0.004, acc = 0.990
Learning Rate 0.00000086 | Epoch 5 | Step 43800 | loss = 0.004, acc = 0.987
Learning Rate 0.00000074 | Epoch 5 | Step 43900 | loss = 0.004, acc = 0.990
Learning Rate 0.00000063 | Epoch 5 | Step 44000 | loss = 0.008, acc = 0.980
Learning Rate 0.00000052 | Epoch 5 | Step 44100 | loss = 0.006, acc = 0.977
Learning Rate 0.00000041 | Epoch 5 | Step 44200 | loss = 0.006, acc = 0.982
Learning Rate 0.00000029 | Epoch 5 | Step 44300 | loss = 0.005, acc = 0.982
Evaluating Dev Set ...


  0%|          | 0/177 [00:00<?, ?it/s]

Validation | Epoch 5 | acc = 0.847
Saving Model ...
Learning Rate 0.00000018 | Epoch 5 | Step 44400 | loss = 0.007, acc = 0.975
Learning Rate 0.00000006 | Epoch 5 | Step 44500 | loss = 0.005, acc = 0.987


## Testing

In [ ]:
print("Evaluating Test Set ...")

result1 = []
result2 = []
result3 = []
# result4 = []
model1.eval()
model2.eval()
model3.eval()
# model4.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output1 = model1(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output2 = model2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output3 = model3(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        # output4 = model4(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
        #                attention_mask=data[2].squeeze(dim=0).to(device))
        result1.append(evaluate(data, output1))
        result2.append(evaluate(data, output2))
        result3.append(evaluate(data, output3))
        # result4.append(evaluate(data, output4))
        # if evaluate(data, output1) == evaluate(data, output2) or evaluate(data, output1) == evaluate(data, output3):
        #   result.append(evaluate(data, output1))
        #   continue
        # elif evaluate(data, output2) == evaluate(data, output3):
        #   result.append(evaluate(data, output2))
        #   continue
        # else:
        #   result.append(evaluate(data, output1))

result_file1 = "mac_result_84.2_1.csv"
with open(result_file1, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result1[i].replace(',','')}\n")
result_file2 = "mac_result_83.6.csv"
with open(result_file2, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result2[i].replace(',','')}\n")
result_file3 = "mac_result_83.1_1.csv"
with open(result_file3, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result3[i].replace(',','')}\n")
      
# result_file4 = "result_88.3_2.csv"
# with open(result_file4, 'w') as f:	
# 	  f.write("ID,Answer\n")
# 	  for i, test_question in enumerate(test_questions):
#         # Replace commas in answers with empty strings (since csv is separated by comma)
#         # Answers in kaggle are processed in the same way
# 		    f.write(f"{test_question['id']},{result4[i].replace(',','')}\n")


print(f"Completed! Result is in {result_file1}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

Completed! Result is in mac_result_84.2_1.csv
